In [0]:
---------
--silver--
----------

CREATE OR REFRESH MATERIALIZED VIEW dlta_silver_mvrh_monthly AS
  WITH base AS (
    SELECT
      `date`,
      `open`,
      `high`,
      `low`,
      `close`,
      `volume`,
      MAX(`volume`) OVER (ORDER BY `date`) AS max_volume_so_far
    FROM dlta_bronze_monthly
    ORDER BY `date`
  ),
  mvrh_flagged AS (
    SELECT *,
      volume = max_volume_so_far AS mvrh
    FROM base
  ),
  add_previous_mvrh AS (
    SELECT *,
      LAG(
        CASE WHEN mvrh THEN volume ELSE NULL END
      ) IGNORE NULLS OVER (ORDER BY `date`) AS last_mvrh_volume
    FROM mvrh_flagged
  ),
  final AS (
    SELECT *,
      CASE 
        WHEN last_mvrh_volume IS NOT NULL THEN 
          ROUND((volume - last_mvrh_volume) / last_mvrh_volume * 100, 2)
        ELSE NULL 
      END AS from_mvrh_percent
    FROM add_previous_mvrh
  )
  SELECT * 
  FROM final

In [0]:
--------
--gold--
--------

CREATE OR REFRESH MATERIALIZED VIEW dlta_gold_ml_mvrh_daily AS
  --CTE 1
  WITH monthly_mvrh AS (
      SELECT
        date AS mvrh_month_date,
        low AS mvrh_low,
        high AS mvrh_high,
        from_mvrh_percent
      FROM dlta_silver_mvrh_monthly
      WHERE mvrh = true
      ORDER BY date
  ),
  --CTE 2
  trade_days AS (
    SELECT
      date,
      ROW_NUMBER() OVER (ORDER BY date) AS trade_day_seq
    FROM dlta_silver_daily
  ),
  --CTE 3
  mvrh_month_last_trade AS (
    SELECT
      DATE_TRUNC('month', date) AS mvrh_month_date,
      MAX(date) AS last_trade_date_of_mvrh_month
    FROM dlta_silver_daily
    GROUP BY DATE_TRUNC('month', date)
  ),
  --CTE 4
  daily_with_prev AS (
    SELECT
      d.date,
      d.close,
      d.60fhr_percent,
      d.60flr_percent,
      mm.from_mvrh_percent,
      mm.mvrh_low,
      mm.mvrh_high,
      mm.mvrh_month_date AS last_mvrh_month_date,
      mmlt.last_trade_date_of_mvrh_month,
      td_d.trade_day_seq AS d_trade_seq,
      td_m.trade_day_seq AS mvrh_trade_seq,
      (td_d.trade_day_seq - td_m.trade_day_seq) AS days_from_mvrh
    FROM dlta_silver_daily d
    LEFT JOIN LATERAL (
      SELECT *
      FROM monthly_mvrh mm
      WHERE mm.mvrh_month_date < DATE_TRUNC('month', d.date)
      ORDER BY mm.mvrh_month_date DESC
      LIMIT 1
    ) mm
    LEFT JOIN mvrh_month_last_trade mmlt
      ON mmlt.mvrh_month_date = mm.mvrh_month_date
    LEFT JOIN trade_days td_d
      ON td_d.date = d.date
    LEFT JOIN trade_days td_m
      ON td_m.date = mmlt.last_trade_date_of_mvrh_month
  ),
  --CTE 5
  filtered_daily AS (
    SELECT *
    FROM daily_with_prev
    WHERE
      date IS NOT NULL
      AND close IS NOT NULL
      AND 60fhr_percent IS NOT NULL
      AND 60flr_percent IS NOT NULL
      AND from_mvrh_percent IS NOT NULL
      AND mvrh_low IS NOT NULL
      AND mvrh_high IS NOT NULL
      AND last_mvrh_month_date IS NOT NULL
  )

  SELECT
    date,
    from_mvrh_percent,
    days_from_mvrh,
    ROUND((close / mvrh_low - 1) * 100, 2) AS from_mvrh_low_percent,
    ROUND((close / mvrh_high - 1) * 100, 2) AS from_mvrh_high_percent,
    (60fhr_percent > 20 AND 60flr_percent > -10) AS 60_days_profitable
  FROM filtered_daily